In [1]:
import re

with open("kaijin_nijumenso.txt", mode="r", encoding="utf-8") as f:
    text_original = f.read()

text = re.sub("《[^》]+》", "", text_original)
text = re.sub("［[^］]+］", "", text)
text = re.sub("[｜ 　]", "", text)
print("文字数", len(text))

文字数 110323


In [2]:
n_rnn = 10
batch_size = 128
epochs = 60
n_mid = 128

In [7]:
import numpy as np

chars = sorted(list(set(text)))
print("文字数(重複なし) ", len(chars))
print(chars[:10])
char_indices = {}
for i, char in enumerate(chars):
    char_indices[char] = i
indices_char = {}
for i, char in enumerate(chars):
    indices_char[i] = char

time_chars = []
next_chars = []
for i in range(0, len(text) - n_rnn):
    time_chars.append(text[i: i+n_rnn])
    next_chars.append(text[i + n_rnn])

x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=bool)
t = np.zeros((len(time_chars), len(chars)), dtype=bool)
for i, t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1

print("xの形状", x.shape)
print("tの形状", t.shape)


文字数(重複なし)  1249
['\n', '—', '…', '、', '。', '々', '〇', '「', '」', '『']
xの形状 (110313, 10, 1249)
tの形状 (110313, 1249)


In [8]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN

model = Sequential()
model.add(SimpleRNN(n_mid, input_shape=(n_rnn, len(chars))))
model.add(Dense(len(chars), activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer="adam")
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               176384    
                                                                 
 dense (Dense)               (None, 1249)              161121    
                                                                 
Total params: 337,505
Trainable params: 337,505
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
from keras.callbacks import LambdaCallback

def on_epoch_end(epoch, logs):
    print("エポック: ", epoch)

    beta = 5
    prev_text = text[0:n_rnn]
    created_text = prev_text

    print("シード: ", created_text)

    for i in range(400):
        x_pred = np.zeros((1, n_rnn, len(chars)))
        for j, char in enumerate(prev_text):
            x_pred[0, j, char_indices[char]] = 1

        y = model.predict(x_pred)
        p_power = y[0] ** beta
        next_index = np.random.choice(len(p_power), p=p_power/np.sum(p_power))
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + next_char
    
    print(created_text)
    print()

epock_end_callback=LambdaCallback(on_epoch_end=on_epoch_end)
    

In [ ]:
history = model.fit(x, t,
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[epock_end_callback])